# 4-3,AutoGraph的使用规范

有三种计算图的构建方式：静态计算图，动态计算图，以及Autograph。

TensorFlow 2.0主要使用的是动态计算图和Autograph。

动态计算图易于调试，编码效率较高，但执行效率偏低。

静态计算图执行效率很高，但较难调试。

而Autograph机制可以将动态图转换成静态计算图，兼收执行效率和编码效率之利。

当然Autograph机制能够转换的代码并不是没有任何约束的，有一些编码规范需要遵循，否则可能会转换失败或者不符合预期。

我们将着重介绍Autograph的编码规范和Autograph转换成静态图的原理。

并介绍使用tf.Module来更好地构建Autograph。

本篇我们介绍使用Autograph的编码规范。

### 一，Autograph编码规范总结


* 1，被@tf.function修饰的函数应尽可能使用TensorFlow中的函数而不是Python中的其他函数。例如使用tf.print而不是print，使用tf.range而不是range，使用tf.constant(True)而不是True.

* 2，避免在@tf.function修饰的函数内部定义tf.Variable. 

* 3，被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等数据结构变量。


### 二，Autograph编码规范解析

 **1，被@tf.function修饰的函数应尽量使用TensorFlow中的函数而不是Python中的其他函数。**

In [7]:
import numpy as np
import tensorflow as tf

@tf.function
def np_random():
    a = np.random.randn(3,3)
    tf.print(a)
    
@tf.function
def tf_random():
    a = tf.random.normal((3,3))
    tf.print(a)

In [8]:
#转为@tf.function后，np_random每次执行都是一样的结果。
np_random()
np_random()

array([[-0.07276738, -0.4568198 , -0.68815881],
       [-0.89726928,  1.23815815, -0.4797338 ],
       [ 0.10697997, -0.05038003, -0.59116474]])
array([[-0.07276738, -0.4568198 , -0.68815881],
       [-0.89726928,  1.23815815, -0.4797338 ],
       [ 0.10697997, -0.05038003, -0.59116474]])


In [3]:
#tf_random每次执行都会有重新生成随机数。
tf_random()
tf_random()

[[1.27245 -0.97735244 0.204800025]
 [0.218641534 -1.83772492 -0.19539158]
 [-1.20412576 0.552548528 -0.0468624309]]
[[1.37561369 0.491967708 -0.986731887]
 [-0.933407843 0.132655337 0.475202322]
 [-0.253822863 -0.588594854 0.692116439]]


**2，避免在@tf.function修饰的函数内部定义tf.Variable.**

In [9]:
# 避免在@tf.function修饰的函数内部定义tf.Variable.

x = tf.Variable(1.0,dtype=tf.float32)
@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return(x)

outer_var() 
outer_var()


2
3


<tf.Tensor: id=40, shape=(), dtype=float32, numpy=3.0>

In [10]:
@tf.function
def inner_var():
    x = tf.Variable(1.0,dtype = tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return(x)

#执行将报错
#inner_var()
#inner_var()


```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-12-c95a7c3c1ddd> in <module>
      7 
      8 #执行将报错
----> 9 inner_var()
     10 inner_var()

~/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/eager/def_function.py in __call__(self, *args, **kwds)
    566         xla_context.Exit()
    567     else:
--> 568       result = self._call(*args, **kwds)
    569 
    570     if tracing_count == self._get_tracing_count():
......
ValueError: tf.function-decorated function tried to create variables on non-first call.
```

**3,被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等结构类型变量。**

In [12]:
tensor_list = []

#@tf.function #加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
tf.print(tensor_list)


[5, 6]


```
[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]
```

In [15]:
tensor_list = []

@tf.function #加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list


append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)


[<tf.Tensor 'x:0' shape=() dtype=float32>]


如果对本书内容理解上有需要进一步和作者交流的地方，欢迎在公众号"Python与算法之美"下留言。作者时间和精力有限，会酌情予以回复。

![image.png](./data/Python与算法之美logo.jpg)